<a href="https://colab.research.google.com/github/ljpetkovic/Charcot_KeyBERT_Keyphrase-Vectorizers/blob/main/KeyBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extraction des mots/phrases-clés avec `keybert` et `keyphrase-vectorizers`
### Approche _PatternRank_
###### [Schopf _et al._, 2022](https://arxiv.org/pdf/2210.05245.pdf)
---

#1️⃣ `keybert`
* _cf._ [Grootendorst (2020)](https://doi.org/10.5281/zenodo.4461265)
* librairie Python pour extraire des mots/phrases-clés les plus similaires à un document en exploitant les plongements BERT<br>
⚠️ on doit spécifier la longueur des n-grammes à extraire, alors que l'on ne sait pas quelle est la longueur optimale ;<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;`keyphrase_ngram_range=(1, 3)` : on veut extraire trois types de séquences : uni-, bi- ou trigrammes <br>
⚠️ la grammaticalité des phrases n'est pas prise en compte (p. ex. « scientifique les planches »)

**_Maximal Marginal Relevance_**

* Afin de diversifier les résultats de l'extraction des mots / phrases-clés, on peut utiliser _Maximal Margin Relevance_ (_MMR_), paramètre également basé sur la similarité cosinus :
 * `use_mmr=True, diversity=[0-1]` (le degré de diversité entre 0 et 1)



 **Mots vides**

 Les listes de mots vides proviennent du vectorizer utilisé avec KeyBERT, et non pas de KeyBERT en soi.

 * `stop_words=None` : si aucune liste ne s'applique
 * `stop_words='french'` : si l'on applique une liste de mots vides en français

In [9]:
!pip install keybert
!pip install nltk
!pip install spacy
import torch
import os
from sentence_transformers import SentenceTransformer
from keybert import KeyBERT
from sklearn.feature_extraction.text import CountVectorizer
import nltk
import spacy

from google.colab import drive
# Monter le Google Drive
drive.mount('/content/drive')

# Initialize the Sentence Transformer Model
sentence_model = SentenceTransformer("distiluse-base-multilingual-cased-v1")
kw_model = KeyBERT(model=sentence_model)

# Download and set up French stop words
## si spaCy

# Load spaCy French model
!python -m spacy download fr_core_news_lg
nlp = spacy.load('fr_core_news_lg')
# Convert spaCy's set of stop words to a list
french_stop_words = list(nlp.Defaults.stop_words)


## si NLTK
# nltk.download('stopwords')
# from nltk.corpus import stopwords
# french_stop_words = stopwords.words('french')

# Initialize CountVectorizer with French stop words
vectorizer = CountVectorizer(ngram_range=(1, 3), stop_words=french_stop_words)

# Assuming Google Drive is mounted and paths are correctly set up
path = '/content/drive/MyDrive/ObTIC/ateliers/extraction_mots_cles/corpus/'
file_name = 'echantillon_charcot.txt'
file_path = '../output/keybert_charcot_output.txt'

# Function to sort keywords
def sort_keywords_by_score(keywords):
    # Sort keywords based on the score in descending order
    return sorted(keywords, key=lambda x: x[1], reverse=True)

# List to store all keywords
all_keywords = []

# Extract keywords from the file
with open(os.path.join(path, file_name), 'r') as myfile:
    raw_data = myfile.readlines()
    start = 0
    end = 20
    while len(raw_data) >= end:
        data = " ".join(raw_data[start:end])
        start = end
        end += 20
        keywords = kw_model.extract_keywords(data, vectorizer=vectorizer, use_mmr=True, diversity=0.7)
        all_keywords.extend(keywords)

# Sort all keywords once after extraction
sorted_keywords = sort_keywords_by_score(all_keywords)

# Write sorted keywords to the output file
with open(os.path.join(path, file_path), 'w') as outfile:
    for keyword, score in sorted_keywords:
        print(f"{keyword}: {score}")
        outfile.write(f"{keyword}: {score}\n")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.8/571.8 MB 2.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['neuf', 'qu', 'quelqu'] not in stop_words.
  warnings.warn(


pyramidal croisé gauche: 0.633
cancer primitif colonne: 0.5774
cellule nucléole prolongement: 0.5757
altérée cordon zone: 0.5622
brain 1890 somnambulismes: 0.5492
voit tumeur située: 0.5424
postérieur corne postérieure: 0.5269
gliomateux tissu nerveux: 0.5198
fibreux coupés transversalement: 0.5182
pyramidal croisé faisceau: 0.5104
localiser anatomie pathologique: 0.5099
scientifique planches reproduction: 0.5093
postérieure cordon postérieur: 0.5078
gauche corne postérieure: 0.4996
corne postérieure: 0.4956
corne corne postérieure: 0.4844
tissu tubes nerveux: 0.4786
région cervicale figure: 0.4778
faisceau pyramidal croisé: 0.476
zone westphal indemne: 0.474
postérieure sillon postérieur: 0.4722
altérée cellules altérées: 0.4699
westphal cordon postérieur: 0.4691
corne corne postérieure: 0.4678
postérieure sillon postérieur: 0.4678
sillon faisceau postérieur: 0.4669
postérieure jt cellule: 0.4638
corne corne postérieure: 0.4636
postérieur faisceaux pyramidaux: 0.4634
postérieure sillo

#2️⃣ PatternRank
* `keybert` + **`keyphrase-vectorizers`** = PatternRank<br>
 ❇️ pas besoin de spécifier la longueur des n-grammes à extraire, car la librairie l'infère elle-même<br>
❇️ la grammaticalité des phrases est prise en compte grâce aux extractions des parties du discours (p. ex. `<N.*>*<ADJ.*>*<ADJ.*>+`--> _sclérose latérale amyotrophique_)
* _cf._ [Schopf _et al._ (2022)](https://arxiv.org/pdf/2210.05245.pdf) et [Schopf (2022)](https://towardsdatascience.com/enhancing-keybert-keyword-extraction-results-with-keyphrasevectorizers-3796fa93f4db)



In [7]:
# !pip install keyphrase-vectorizers
# !pip install flair

# !pip install keybert
from keyphrase_vectorizers import KeyphraseCountVectorizer
from keybert import KeyBERT
from flair.embeddings import TransformerDocumentEmbeddings
import os
import spacy
# from google.colab import drive

# Mount Google Drive
# drive.mount('/content/drive')

# Load the spaCy model
# !python -m spacy download fr_core_news_lg
# nlp = spacy.load("fr_core_news_lg")

# Set paths
path = '/content/drive/MyDrive/ObTIC/ateliers/extraction_mots_cles/corpus/'
input_file_name = 'echantillon_charcot.txt'
output_file_name = '../output/charcot_output.txt'

# Initialize KeyBERT with a multilingual model
kw_model = KeyBERT(model=TransformerDocumentEmbeddings('bert-base-multilingual-cased'))

# Setup vectorizer with specific language and pattern settings
vectorizer = KeyphraseCountVectorizer(spacy_pipeline=nlp, pos_pattern='<N.*>+<ADJ.*>*', stop_words='french')

# Function to sort keywords by score
def sort_keywords_by_score(keywords):
    return sorted(keywords, key=lambda x: x[1], reverse=True)

# List to store all keywords
all_keywords = []

# Read and process the file
with open(os.path.join(path, input_file_name), 'r') as input_file:
    raw_data = input_file.readlines()
    start = 0
    end = 20  # divide text into chunks
    while start < len(raw_data):
        data = " ".join(raw_data[start:end]).replace('\n', ' ')
        start = end
        end += 20
        try:
            # Extract keyphrases
            kp = kw_model.extract_keywords(data, vectorizer=vectorizer)
            all_keywords.extend(kp)
        except ValueError as e:
            print(f"An error occurred while processing chunks starting at line {start}: {e}")

# Sort all keywords at the end
sorted_keywords = sort_keywords_by_score(all_keywords)

# Write sorted keywords to output file
with open(os.path.join(path, output_file_name), 'w') as output_file:
    for keyword, score in sorted_keywords:
        print(f"{keyword}: {score}")
        output_file.write(f"{keyword}: {score}\n")


c. trousseaux fibreux: 0.9306
h. cordon antérieur sain: 0.9284
k. faisceau: 0.9226
c. tissu gliomateux: 0.9208
tissu gingival: 0.9184
.perfec-: 0.918
- thyroïdien: 0.915
a. cellule multipolaire: 0.9147
g. tumeur: 0.9147
c. racine postérieure: 0.9139
d. racine postérieure: 0.9111
érysipèle périodique annuel: 0.9109
travaux récents: 0.91
c. racine postérieure: 0.9091
c. racine postérieure: 0.9087
faisceau pyramidal direct: 0.9064
k. faisceau cérébelleux direct: 0.9056
k. sclérose latérale gauche: 0.9053
j. partie saine: 0.9052
foie: 0.9052
syringomyélie: 0.9048
b. tubes nerveux persistant: 0.9047
foie: 0.9036
l. faisceau postérieur sclérosé: 0.9033
c. tube nerveux: 0.9033
7j. faisceau antérieur: 0.902
rétractions fibro: 0.9008
goitre exophtalmique: 0.9008
j. partie sclérosée: 0.9
e. faisceau latéral droit: 0.9
c. racine postérieure: 0.8996
fibrome: 0.8995
c. faisceau antérieur: 0.8993
faisceau pyramidal: 0.899
e. cordon postérieur: 0.8988
a. faisceaux scléreux: 0.8985
d. tissu nerveux sa

# 📡 Repérage des phrases-clés communes

In [ ]:
# Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')
import re
import numpy as np
pattern = re.compile(r":.*\n")
charcot_pr = "/content/drive/MyDrive/ObTIC/ateliers/extraction_mots_cles/output/output_pattern_rank_charcot_sorted_20.txt"
autres_pr = "/content/drive/MyDrive/ObTIC/ateliers/extraction_mots_cles/output/output_pattern_rank_autres_sorted_20.txt"



# N'extraire que des phrases-clés, sans leurs scores
with open(charcot_pr, 'r') as input_file_charcot, open(autres_pr, 'r') as input_file_autres:
    raw_data_charcot = input_file_charcot.readlines()
    raw_data_autres = input_file_autres.readlines()
    res_charcot = [pattern.sub("", match) for match in raw_data_charcot]
    res_autres = [pattern.sub("", match) for match in raw_data_autres]
    # for r in res_charcot:
      # print(r)
    # for r2 in res_autres:
    #   print(r2)

    common_elements = np.intersect1d(res_charcot, res_autres)
    celem_list = common_elements.tolist()
    for c in celem_list:
      print(c)



cordons
deux moitiés
foie
hypnotisme
observation iv
planche ii
planche iv
planche ix
planche vii
planche viii
planche xi
planche xii
planche xiii
planche xiv
planche xvi
planche xvii
planche xxi
planche xxiii
région lombaire
